In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pydantic.dataclasses import dataclass, Field
import dotenv
import interlab
import langchain
from interlab import actor
from treetrace import TracingNode
from interlab.queries.web_console import WebConsoleModel
from treetrace.ext.pyplot import capture_figure
from interlab.environment.experimental.commgame import CommunicateAndPlayGame
from interlab.environment.experimental.tree import expand_tree
from matplotlib import pyplot as plt
dotenv.load_dotenv()

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
#import langchain
#from langchain.cache import InMemoryCache
#langchain.llm_cache = InMemoryCache()

In [ ]:
console_actor = actor.WebConsoleActor("MyConsoleActor")
console_actor

In [ ]:
PROMPT = """You are {name}. We are playing a repeated game where each player may choose one of the following actions: "cooperate" or "defect".
Before the each we can simulatenously send a message to each other. After that we both secretely choose one of action.
If we both play "cooperate" then we get $3 each. If I play "defect" and the opponent plays "cooperate" then I get $5 and they get nothing. If they play "defect" when I play "cooperate" then they get $5 and I get nothing.
If we both play "defect" then we get $1 each.
"""

e35 = langchain.chat_models.ChatOpenAI(model_name='gpt-3.5-turbo')

pa = actor.OneShotLLMActor("Alice", e35, PROMPT.format(name="Alice"))
pb = actor.OneShotLLMActor("Bob", e35, PROMPT.format(name="Bob"))
#pb = console_actor

actions = ["cooperate", "defect"]

payoffs = np.array(
    [
        [[3, 3], [0, 5]],
        [[5, 0], [1, 1]],
    ]
)

In [ ]:
game = CommunicateAndPlayGame(actors=[pa, pb], n_rounds=3, action_names=actions, payoff_matrix=payoffs)
plt.ioff()
with TracingNode("game") as ctx:
    while not game.is_finished:
        game.step()
    ctx.set_result({
        "history": game.history,
        "payoffs": capture_figure(game.payoff_chart(cumsum=True))
    })
ctx.display()

In [ ]:
game.history

In [ ]:
env = CommunicateAndPlayGame(actors=[pa, pb], n_rounds=2, action_names=actions, payoff_matrix=payoffs)

with TracingNode("expand tree demo") as ctx:
    expand_tree(env, max_depth=100, n_children=3)
ctx.display()

In [ ]:
ctx.display()